# Identify the consititutive heterochromatin states among the mouse genomce by H3K9me3 signals

In [2]:
suppressPackageStartupMessages({
    library(tidyverse)
    library(GenomicRanges)
    library(rtracklayer)
    library(tmpRpkg)
})
Sys.setenv("_R_USE_PIPEBIND_" = TRUE)
# change the size of figures shown in jupyter
options(repr.plot.width=10, repr.plot.height=8)

In [3]:
# * meta# * meta
projd <- here::here()
binSize <- 5000
genomeBind <- file.path(projd, "09.epimem", "out", "genomeBin")
## load blacklist
blfnm <- file.path(projd, "meta", "mm10-blacklist.v2.bed")
blgr <- loadGRfromBed(blfnm, header = TRUE)
blgr

GRanges object with 3434 ranges and 1 metadata column:
         seqnames            ranges strand | High Signal Region
            <Rle>         <IRanges>  <Rle> |        <character>
     [1]    chr10   3218900-3276600      * |    Low Mappability
     [2]    chr10   3576900-3627700      * |    Low Mappability
     [3]    chr10   4191100-4197600      * |    Low Mappability
     [4]    chr10   4613500-4615400      * | High Signal Region
     [5]    chr10   4761300-4763900      * | High Signal Region
     ...      ...               ...    ... .                ...
  [3430]     chrY   6530200-6663200      * | High Signal Region
  [3431]     chrY   6760200-6835800      * | High Signal Region
  [3432]     chrY   6984100-8985400      * | High Signal Region
  [3433]     chrY 10638500-41003800      * | High Signal Region
  [3434]     chrY 41159200-91744600      * | High Signal Region
  -------
  seqinfo: 21 sequences from an unspecified genome; no seqlengths

In [4]:
# load gtf
mm10 <- import(file.path(projd, "meta", "gencode.vM25.annotation.gtf"))
mm10
table(mm10$gene_type)

GRanges object with 1872052 ranges and 21 metadata columns:
            seqnames          ranges strand |   source       type     score
               <Rle>       <IRanges>  <Rle> | <factor>   <factor> <numeric>
        [1]     chr1 3073253-3074322      + |  HAVANA  gene              NA
        [2]     chr1 3073253-3074322      + |  HAVANA  transcript        NA
        [3]     chr1 3073253-3074322      + |  HAVANA  exon              NA
        [4]     chr1 3102016-3102125      + |  ENSEMBL gene              NA
        [5]     chr1 3102016-3102125      + |  ENSEMBL transcript        NA
        ...      ...             ...    ... .      ...        ...       ...
  [1872048]     chrM     15289-15355      + |  ENSEMBL transcript        NA
  [1872049]     chrM     15289-15355      + |  ENSEMBL exon              NA
  [1872050]     chrM     15356-15422      - |  ENSEMBL gene              NA
  [1872051]     chrM     15356-15422      - |  ENSEMBL transcript        NA
  [1872052]     chrM     153


          3prime_overlapping_ncRNA                          antisense 
                                16                              19316 
     bidirectional_promoter_lncRNA                          IG_C_gene 
                              1211                                246 
                   IG_C_pseudogene                          IG_D_gene 
                                 5                                 78 
                   IG_D_pseudogene                          IG_J_gene 
                                 9                                 64 
                        IG_LV_gene                      IG_pseudogene 
                                22                                  7 
                         IG_V_gene                    IG_V_pseudogene 
                              2000                                587 
                           lincRNA                       macro_lncRNA 
                             40389                                 10 
     

In [5]:
## get TE annotations in the genome
loadRepeatMasker <- function() {
    colnms <- c("bin", "swScore", "milliDiv", "milliDel", "milliIns", "genoName", "genoStart", "genoEnd", 
               "genoLeft", "strand", "repName", "repClass", "repFamliy", "repStart", "repEnd", "repLeft", 
               "id")
    r <- data.table::fread(
        file = file.path(projd, "meta", "mm10.RepeatMasker.txt"), 
        sep = "\t", header = FALSE) |>
      setNames(object = _, nm = colnms)
    return(r)
}
rawTEs <- loadRepeatMasker()
## other heterochromatin annotations

In [6]:
head(rawTEs)

bin,swScore,milliDiv,milliDel,milliIns,genoName,genoStart,genoEnd,genoLeft,strand,repName,repClass,repFamliy,repStart,repEnd,repLeft,id
<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>
585,463,13,6,17,chr1,10000,10468,-248945954,+,(TAACCC)n,Simple_repeat,Simple_repeat,1,471,0,1
585,3612,114,215,13,chr1,10468,11447,-248944975,-,TAR1,Satellite,telo,-399,1712,483,2
585,484,251,132,0,chr1,11504,11675,-248944747,-,L1MC5a,LINE,L1,-2382,395,199,3
585,239,294,19,10,chr1,11677,11780,-248944642,-,MER5B,DNA,hAT-Charlie,-74,104,1,4
585,318,230,37,0,chr1,15264,15355,-248941067,-,MIR3,SINE,MIR,-119,143,49,5
585,18,232,0,19,chr1,15797,15849,-248940573,+,(TGCTCC)n,Simple_repeat,Simple_repeat,1,52,0,6


In [11]:
# * functions
histbwd <- file.path(projd, "09.epimem", "out", "genomeBin")
loadAllHistoneSignals <- function(sc) {
  r1 <- read.csv(file.path(histbwd, paste0(sc, ".H3K9me3.5000.csv")), sep = ",", header = TRUE)
  rownames(r1) <- r1$region
  r2 <- read.csv(file.path(histbwd, paste0(sc, ".H3K27ac_H3K27me3_H3K4me1.5000.csv")), sep = ",", header = TRUE)
  rownames(r2) <- r2$region
  r <- merge(r1, r2, by = "region", all = FALSE)
  return(r)
}

In [13]:
# analyze one k9 as example
sc <- "326_OPC_NN"
k9 <- loadAllHistoneSignals(sc)
head(k9)
max(k9$H3K9me3)
quantile(k9$H3K9me3, probs = c(0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 0.999))

,region,H3K9me3,H3K27ac,H3K27me3,H3K4me1
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,chr1:1-5000,0.0000000,0.0000000,0.0000000,0.0000000
2,chr1:100000001-100005000,1.3969261,0.3557545,0.9780488,0.3610463
3,chr1:10000001-10005000,0.1817366,0.3313600,1.1783989,0.2195246
4,chr1:1000001-1005000,0.0000000,0.0000000,0.0000000,0.0000000
5,chr1:100001-105000,0.0000000,0.0000000,0.0000000,0.0000000
6,chr1:100005001-100010000,0.6263849,0.2784645,0.7742647,0.2754316


[1] 90.97491

10%       25%       50%       75%       90%       95%       99%     99.9% 
0.1874992 0.3075027 0.5095049 0.8720211 1.5129094 2.0468474 3.5614128 5.8035416

In [ ]:
# remove blacklist and sex chromosomes


In [ ]:
# The regions should have low H3K27ac, H3K27me3, H3K4me1 signals (at least lower than Q50)